In [1]:
from skimage.io import imread_collection
from skimage.transform import resize
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

def load_images(class_name, values, labels):
    path_to_image = 'raw-img/'
    postfix = '/*.jpeg'
    image_size = (128,128)
    for image in imread_collection(path_to_image + class_name + postfix):
        values.append(resize(image, image_size))
        labels.append(class_name)

def create_data_set():
    class_names = ['spider', 'horse', 'elephant', 'chicken', 'sheep']
    values = []
    labels = []
    for class_name in class_names:
        load_images(class_name, values, labels)

    return np.array(values), np.array(labels)

x, y = create_data_set()
 
class HogTransform(BaseEstimator, TransformerMixin):
    def __init__(self, y=None, orientations=9,
                 pixels_per_cell=(8, 8),
                 cells_per_block=(3, 3), block_norm='L2-Hys'):
        self.y = y
        self.orientations = orientations
        self.pixels_per_cell = pixels_per_cell
        self.cells_per_block = cells_per_block
        self.block_norm = block_norm
 
    def fit(self, X, y=None):
        return self
 
    def transform(self, X, y=None):
        return np.array([ 
            hog(img,
                orientations=self.orientations,
                pixels_per_cell=self.pixels_per_cell,
                cells_per_block=self.cells_per_block, channel_axis=-1) for img in X ])


/home/wiktor/.local/lib/python3.8/site-packages/skimage/io/manage_plugins.py:23: UserWarning: Your installed pillow version is < 8.1.2. Several security issues (CVE-2021-27921, CVE-2021-25290, CVE-2021-25291, CVE-2021-25293, and more) have been fixed in pillow 8.1.2 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper
/home/wiktor/.local/lib/python3.8/site-packages/skimage/io/manage_plugins.py:293: UserWarning: Your installed pillow version is < 8.1.2. Several security issues (CVE-2021-27921, CVE-2021-25290, CVE-2021-25291, CVE-2021-25293, and more) have been fixed in pillow 8.1.2 or higher. We recommend to upgrade this library.
  plugin_module = __import__('skimage.io._plugins.' + modname,


In [ ]:
hog_pipeline = Pipeline([
    ('hog_transform', HogTransform()),
    ('scaler', StandardScaler()),
    ('logistic',KNeighborsClassifier(n_neighbors=6))
])
 
param_grid = [
    {
        'hog_transform__orientations': [7, 8, 9, 10, 11],
        'hog_transform__cells_per_block': [(2, 2), (3, 3), (4,4), (5,5), (6,6), (7,7), (8,8)],
        'hog_transform__pixels_per_cell': [(6,6), (7,7), (8, 8), (9,9), (10, 10), (12, 12)]
    },

]
search = GridSearchCV(hog_pipeline, param_grid, n_jobs=2, verbose=10)
search.fit(x, y)
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)


In [8]:
from sklearn import svm

scalify = StandardScaler()
hog_trasform = HogTransform(orientations=11, cells_per_block=(7,7), pixels_per_cell=(8,8))

x_train = scalify.fit_transform(hog_trasform.transform(x))

svm_pipeline = Pipeline([
    ('logistic',svm.SVC())
])
 
param_grid = [
    {
        'logistic__C': [ 0.4, 0.5, 0.8, 1.1, 2, 3, 5, 6]
    },

]
search = GridSearchCV(svm_pipeline, param_grid, n_jobs=2, verbose=10)
search.fit(x_train, y)
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/wiktor/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3369, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_25659/3118681098.py", line 6, in <cell line: 6>
    x_train = scalify.fit_transform(hog_trasform.transform(x))
  File "/tmp/ipykernel_25659/3347403700.py", line 44, in transform
    return np.array([
  File "/tmp/ipykernel_25659/3347403700.py", line 45, in <listcomp>
    hog(img,
  File "/home/wiktor/.local/lib/python3.8/site-packages/skimage/_shared/utils.py", line 406, in fixed_func
    return func(*args, **kwargs)
  File "/home/wiktor/.local/lib/python3.8/site-packages/skimage/_shared/utils.py", line 348, in fixed_func
    return func(*args, **kwargs)
  File "/home/wiktor/.local/lib/python3.8/site-packages/skimage/feature/_hog.py", line 293, in hog
    _hog_normalize_block(block, method=block_norm)
  File "/home/wiktor/.local/lib/python3.8/site-packages/skimage